<html>
    <div dir = "rtl" style="font-size: 26px; color: blue">
        <b>
            پروژه دوم درس هوش مصنوعی
            <br>
            <br>
            Game
        </b>
    </div>
<br>
<br>
<br>
    <div dir = "rtl" style="font-size: 24px; color: gray">
        <b>
            معین کرمی
            <br>
            <br>
            810198540
        </b>
    </div>
</html>

<html>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            هدف پروژه:
        </b>
    </div>
    <div dir = "rtl" style = "font-size: 16px">
        <b>
            <br>
            هدف این پروژه آشنایی با انواع الگوریتم برای حل 
            Adversarial Games
            مانند الگوریتم
            min-max
            و همچنین تکنیک هایی مانند هرس آلفا بتا است.
        </b>
        <br>
        <br>
    </div>
</html>

<html>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            خلاصه ی مسئله:
        </b>
    </div>
    <br>
    <div dir = "rtl" style = "font-size: 16px">
            در این مسئله یک جدول داریم (که سایز استاندارد آن 
        $6 \times 7$
        است). در هر مرحله هر فرد که نوبتش است یک ستون را انتخاب میکند و مهره ی خود را داخل پایین ترین خانه ی خالی آن ستون قرار می‌دهد. اگر در یک مرحله ۴ مهره یکسان به صورت افقی، عمودی و یا مورب مجاور هم قرار بگیرند، فردی که این مهره ها برای اوست برنده میشود.
        در نهایت اگر جدول پر شود و هیچ ۴ مهره مجاور یکسانی نداشته باشیم بازی مساوی می‌شود.
        <br>
        هدف ما پیاده سازی یک استراتژی بر اساس
        min-max
        است.
    </div>
</html>


In [1]:
from random import random
import copy
import time

In [3]:
global max_search_depth
global pruning
global seen_nodes
global state_score
state_score = dict()
global seen_states

class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        if pruning:
            return pruning_maximize(self, max_search_depth, float('inf'), -float('inf'))[1]
        else:
            return normal_maximize(self, max_search_depth)[1]
        raise NotImplementedError

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()

<html>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            پیاده سازی 
            min-max
            بدون هرس آلفا بتا
        </b>
    </div>
</html>

In [4]:
def normal_maximize(game, remaining_depth):
    seen_nodes[0] += 1
    res = game.check_for_winners()
    if remaining_depth == 0 or res != 0:
        return (calculate_score(game), None)
    
    if (str(game.board), remaining_depth, game.YOU) in seen_states:
        return seen_states[(str(game.board), remaining_depth, game.YOU)]
    
    possible_moves = game.get_possible_moves()
    max_score = -float('inf')

    if len(possible_moves) == 0:
        return (0, None)
    for move in possible_moves:
        tmp_game = copy.deepcopy(game)
        tmp_game.register_input(move, tmp_game.YOU)
        score = normal_minimize(tmp_game, remaining_depth - 1)[0]
        if score >= max_score:
            max_score, final_move = score, move
    
    seen_states[(str(game.board), remaining_depth, game.YOU)] = (max_score, final_move)
    return (max_score, final_move)

def normal_minimize(game, remaining_depth):
    seen_nodes[0] += 1
    res = game.check_for_winners()
    if remaining_depth == 0 or res != 0:
        return (calculate_score(game), None)
    
    if (str(game.board), remaining_depth, game.CPU) in seen_states:
        return seen_states[(str(game.board), remaining_depth, game.CPU)]
    
    min_score = float('inf')
    possible_moves = game.get_possible_moves()
    
    if len(possible_moves) == 0:
        return (0, None)
    for move in possible_moves:
        tmp_game = copy.deepcopy(game)
        tmp_game.register_input(move, tmp_game.CPU)
        score = normal_maximize(tmp_game, remaining_depth - 1)[0]
        if score <= min_score:
            min_score, final_move = score, move
    seen_states[(str(game.board), remaining_depth, game.CPU)] = (min_score, final_move)
    return (min_score, final_move)

<html>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            تعریف
            heuristic
        </b>
    </div>
    <br>
    <div dir = "rtl" style = "font-size: 16px">
            اگر در این استیت بازی را باخته باشیم مقدار
        $-inf$
        و اگر برده باشیم مقدار 
        $inf$
        و اگر بازی مساوی شده و به اتمام رسیده باشد
        $0$
        را برمی‌گردانیم.
        <br>
        در غیر این صورت به شکل زیر به هر استیت یک امتیاز می‌دهیم: (توجه کنید در همه این حالات اگر مهره ها برای ما باشد، امتیاز مثبت و اگر برای حریف باشد، امتیاز منفی محاسبه می‌شود.
        <br>
        ۱-  اگر دو مهره مجاور یکسان داشته باشیم:
        <br>
        ۱-۱- اگر در هر طرف دو مهره یک جای خالی موجود باشد و یا در یک طرف آن دو جای خالی موجود باشد، 1 امتیاز محاسبه می‌شود.
        <br>
        ۱-۲- اگر در یک سمت آن یک جای خالی و در سمت دیگر آن دو جای خالی موجود باشد، 2 امتیاز محاسبه می‌شود.
        <br>
        ۱-۳- اگر در هر سمت آن دو جای خالی باشد، 3 امتیاز محاسبه می‌شود.
        <br><br>
        ۲- اگر سه مهره مجاور یکسان داشته باشیم:
        <br>
        ۲-۱- اگر تنها در یک طرف آن جای خالی داشته باشیم 6 امتیاز محاسبه می‌شود
        <br>
        ۲-۲- اگر در هر دو طرف آن جای خالی داشته باشیم 10 امتیاز محاسبه می‌شود.
    </div>
</html>

In [5]:

def calculate_score(game):
    if str(game.board) in state_score:
        return state_score[str(game.board)]

    winner = game.check_for_winners()
    if winner:
        return float('inf') * winner
    state_score[str(game.board)] = calculate_horizantally_score(game) + calculate_vertically_score(game) + calculate_diagonally_score(game)
    return state_score[str(game.board)]

def calculate_empty_cells(game, row, col, x_dir, y_dir):
    row += x_dir
    col += y_dir
    res = 0
    while row >= 0 and row < game.rows and col >= 0 and col < game.columns:
        if game.board[row][col] != game.EMPTY:
            break
        res += 1
        row += x_dir
        col += y_dir
    return min(res, 2)

def calculate_horizantally_score(game): 
    score = 0
    for row in range(game.rows):
        for col in range(game.columns):
            if game.board[row][col]:
                left_empty = calculate_empty_cells(game, row, col, 0, -1)
                same_in_row = 1
                while col < game.columns - 1 and game.board[row][col] == game.board[row][col + 1]:
                    same_in_row += 1
                    col += 1
                if same_in_row < 2:
                    continue
                right_empty = calculate_empty_cells(game, row, col, 0, 1)

                if same_in_row == 2:
                    if right_empty + left_empty > 1:
                        score += game.board[row][col] * (right_empty + left_empty - 1)
                else:
                    if right_empty and left_empty:
                        score += game.board[row][col] * 10
                    elif right_empty or left_empty:
                        score += game.board[row][col] * 6
    
    return score

def calculate_vertically_score(game):
    score = 0
    for col in range(game.columns):
        for row in range(game.rows):
            if game.board[row][col]:
                up_empty = calculate_empty_cells(game, row, col, -1, 0)
                same_in_col = 1
                while row < game.rows - 1 and game.board[row][col] == game.board[row + 1][col]:
                    same_in_col += 1
                    row += 1
                if same_in_col < 2:
                    continue
                down_empty = calculate_empty_cells(game, row, col, 1, 0)

                if same_in_col == 2:
                    if up_empty + down_empty > 1:
                        score += game.board[row][col] * (up_empty + down_empty - 1)
                else:
                    if down_empty and up_empty:
                        score += game.board[row][col] * 10
                    elif down_empty or up_empty:
                        score += game.board[row][col] * 6
    return score

def calculate_diagonally_score(game):
    return calculate_right_dig_score(game) + calculate_left_dig_score(game)

def calculate_left_dig_score(game):
    score = 0
    for dig_sum in range(2, game.rows + game.columns - 3):
        row = max(0, dig_sum - game.columns + 1)
        col = dig_sum - row
        while row < game.rows and col > 0:
            if game.board[row][col]:
                right_up_empty = calculate_empty_cells(game, row, col, -1, 1)
                same_in_dig = 1;
                while row < game.rows - 1 and col > 0 and game.board[row][col] == game.board[row + 1][col - 1]:
                    same_in_dig += 1
                    row += 1
                    col -= 1
                if same_in_dig < 2:
                    row += 1
                    col -= 1
                    continue
                left_down_empty = calculate_empty_cells(game, row, col, 1, -1)
                
                if same_in_dig == 2:
                    if right_up_empty + left_down_empty > 1:
                        score += game.board[row][col] * (right_up_empty + left_down_empty - 1)
                else:
                    if right_up_empty and left_down_empty:
                        score += game.board[row][col] * 10
                    elif right_up_empty or left_down_empty:
                        score += game.board[row][col] * 6
            row += 1
            col -= 1
    return score

def calculate_right_dig_score(game):
    score = 0
    for dig_dif in range(-game.columns + 3, game.rows - 2):
        row = max(0, dig_dif)
        col = row - dig_dif
        while row < game.rows and col < game.columns:
            if game.board[row][col]:
                left_up_empty = calculate_empty_cells(game, row, col, -1, -1)
                same_in_dig = 1;
                while row < game.rows - 1 and col < game.columns -1 and game.board[row][col] == game.board[row + 1][col + 1]:
                    same_in_dig += 1
                    row += 1
                    col += 1
                if same_in_dig < 2:
                    row += 1
                    col += 1
                    continue
                right_down_empty = calculate_empty_cells(game, row, col, 1, 1)
                if same_in_dig == 2:
                    if right_down_empty + left_up_empty > 1:
                        score += game.board[row][col] * (right_down_empty + left_up_empty - 1)
                else:
                    if left_up_empty and right_down_empty:
                        score += game.board[row][col] * 10
                    elif left_up_empty or right_down_empty:
                        score += game.board[row][col] * 6
            row += 1
            col += 1
    return score

In [ ]:
board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]
pruning = False
for board in board_sizes_to_check:
    state_score = dict()
    seen_states = dict()
    for depth in range(1, 6, 2):
        max_search_depth = depth
        game = ConnectSin(board_size = board, silent = True)
        win = 0
        seen_nodes = [0]
        
        max_game = dict()
        min_game = dict()
        
        start_time = time.time()
        for i in range(200):
            if game.run() == 1:
                win += 1
        end_time = time.time()
        
        mean_exec_time = (end_time - start_time) / 200
        print(f'board size: {board}\ndepth search: {depth}\nmean execution time: {mean_exec_time}\nwinning chance: {win/200}\nmean number of seen nodes: {seen_nodes[0] / 200}\n')

<table style = "width: 85%; font-size: 14px; margin-left:auto; margin-right:auto;">
    <tr style = "text-align: left">
        <th style = "text-align: left">Test</th>
        <th style = "text-align: left">Search Depth</th>
        <th style = "text-align: left">Board Size</th>
        <th style = "text-align: left">Mean Execution Time</th>
        <th style = "text-align: left">Winning Chance</th>
        <th style = "text-align: left">Mean Seen States</th>
    </tr>
    <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left">0.005</td>
        <td style = "text-align: left">0.91</td>
        <td style = "text-align: left">12.8</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left"> 2.840</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">15310</td>
    </tr>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left"> 0.661</td>
        <td style = "text-align: left"> 0.99</td>
        <td style = "text-align: left">15310.63</td>
    </tr>
    <tr>
        <td style = "text-align: left">4</td>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 0.008</td>
        <td style = "text-align: left"> 0.935</td>
        <td style = "text-align: left">15.015</td>
    </tr>
    <tr>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 0.391</td>
        <td style = "text-align: left"> 0.995</td>
        <td style = "text-align: left">1401.895</td>
    </tr>
    <tr>
        <td style = "text-align: left">6</td>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 8.291</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">33729.48</td>
    </tr>
    <tr>
        <td style = "text-align: left">7</td>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 0.010</td>
        <td style = "text-align: left"> 0.98</td>
        <td style = "text-align: left">14.28</td>
    </tr>
    <tr>
        <td style = "text-align: left">8</td>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 1.02</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">3040.04</td>
    </tr>
    <tr>
        <td style = "text-align: left">9</td>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 24.138</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">75514.63</td>
    </tr>
</table>

<html>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            پیاده سازی 
            min-max
            همراه هرس آلفا بتا
        </b>
    </div>
</html>

In [8]:
from random import shuffle

def arrange_moves(game, possible_moves):
    tmp = []
    for i in possible_moves:
        tmp.append((abs(game.columns / 2 - i), i))
    tmp.sort()
    possible_moves = []
    for i in tmp:
        possible_moves.append(i[1])
    return possible_moves

def pruning_maximize(game, remaining_depth, min_val, max_val):
    seen_nodes[0] += 1
    res = game.check_for_winners()
    if remaining_depth == 0 or res != 0:
        return (calculate_score(game), None)
    
    if (str(game.board), remaining_depth, game.YOU) in seen_states:
        return seen_states[(str(game.board), remaining_depth, game.YOU)]
    
    possible_moves = game.get_possible_moves()
    max_score = -float('inf')

    possible_moves = arrange_moves(game, possible_moves)
    
    for move in possible_moves:
        tmp_game = copy.deepcopy(game)
        tmp_game.register_input(move, tmp_game.YOU)
        score = normal_minimize(tmp_game, remaining_depth - 1)[0]
        if score >= max_score:
            max_score, final_move = score, move
        
        if max_score >= min_val:
            seen_states[(str(game.board), remaining_depth, game.YOU)] = (max_score, final_move)
            return (max_score, final_move)
        max_val = max(max_val, max_score)
    
    seen_states[(str(game.board), remaining_depth, game.YOU)] = (max_score, final_move)
    return (max_score, final_move)

def pruning_minimize(game, remaining_depth, mean_val, max_val):
    seen_nodes[0] += 1
    res = game.check_for_winners()
    if remaining_depth == 0 or res != 0:
        return (calculate_score(game), None)
    
    if (str(game.board), remaining_depth, game.CPU) in seen_states:
        return seen_states[(str(game.board), remaining_depth, game.CPU)]
    
    min_score = float('inf')
    possible_moves = game.get_possible_moves()
    shuffle(possible_moves)
    
    for move in possible_moves:
        tmp_game = copy.deepcopy(game)
        tmp_game.register_input(move, tmp_game.CPU)
        score = normal_maximize(tmp_game, remaining_depth - 1)[0]
        if score <= min_score:
            min_score, final_move = score, move
        
        if min_score <= max_val:
            seen_states[(str(game.board), remaining_depth, game.CPU)] = (min_score, final_move)
            return (min_score, final_move)
        min_val = min(min_val, min_score)
    
    seen_states[(str(game.board), remaining_depth, game.CPU)] = (min_score, final_move)
    return (min_score, final_move)

<html>
    <div dir = "rtl" style = "font-size: 16px">
            در اینجا به دلیل محدودیت زمانی هر بازی را 20 بار اجرا می‌کنیم، توجه داشته باشید چون بازی کمتر اجرا می‌شود در نتیجه استیت های تکراری کمتری می‌بینیم و میانگین زمان اجرا و تعداد استیت دیده شده افزایش می‌یابد ولی میانگین این مقدار برای ۲۰ اجرای اول کمتر از راه بدون هرس کردن است. 
    </div>
</html>

In [9]:
board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]
pruning = True
for board in board_sizes_to_check:
    state_score = dict()
    seen_states = dict()
    for depth in range(1, 8, 2):
        max_search_depth = depth
        game = ConnectSin(board_size = board, silent = True)
        win = 0
        seen_nodes = [0]

        start_time = time.time()
        for i in range(10):
            if game.run() == 1:
                win += 1
        end_time = time.time()
        
        mean_exec_time = (end_time - start_time) / 10
        print(f'board size: {board}\ndepth search: {depth}\nmean execution time: {mean_exec_time}\nwining chance: {win/10}\nmean number of seen nodes: {seen_nodes[0] / 10}\n')

board size: (6, 7)
depth search: 1
mean execution time: 0.012595200538635254
wining chance: 1.0
mean number of seen nodes: 25.6

board size: (6, 7)
depth search: 3
mean execution time: 0.41201319694519045
wining chance: 1.0
mean number of seen nodes: 1933.7

board size: (6, 7)
depth search: 5
mean execution time: 4.416846537590027
wining chance: 1.0
mean number of seen nodes: 26891.1



KeyboardInterrupt: 

<table style = "width: 85%; font-size: 14px; margin-left:auto; margin-right:auto;">
    <tr style = "text-align: left">
        <th style = "text-align: left">Test</th>
        <th style = "text-align: left">Search Depth</th>
        <th style = "text-align: left">Board Size</th>
        <th style = "text-align: left">Mean Execution Time</th>
        <th style = "text-align: left">Winning Chance</th>
        <th style = "text-align: left">Mean Seen States</th>
    </tr>
    <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left">0.008</td>
        <td style = "text-align: left">1.0</td>
        <td style = "text-align: left">21.6</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left"> 0.283</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">1509.4</td>
    </tr>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left"> 2.968</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">18259.1</td>
    </tr>
    <tr>
        <td style = "text-align: left">4</td>
        <td style = "text-align: left">7</td>
        <td style = "text-align: left">$6 \times 7$</td>
        <td style = "text-align: left"> 50.758</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">290206.5</td>
    </tr>
    <tr>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 0.011</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">25.4</td>
    </tr>
    <tr>
        <td style = "text-align: left">6</td>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 0.415</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">1419.3</td>
    </tr>
    <tr>
        <td style = "text-align: left">7</td>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 13.734</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">53170.0</td>
    </tr>
    <tr>
        <td style = "text-align: left">8</td>
        <td style = "text-align: left">7</td>
        <td style = "text-align: left">$7 \times 8$</td>
        <td style = "text-align: left"> 203.545</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">871297.4</td>
    </tr>
    <tr>
        <td style = "text-align: left">9</td>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 0.019</td>
        <td style = "text-align: left"> 0.9</td>
        <td style = "text-align: left">37.1</td>
    </tr>
    <tr>
        <td style = "text-align: left">10</td>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 1.487</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">4827.7</td>
    </tr>
    <tr>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">5</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 50.831</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">180637.5</td>
    </tr>
    <tr>
        <td style = "text-align: left">12</td>
        <td style = "text-align: left">7</td>
        <td style = "text-align: left">$7 \times 10$</td>
        <td style = "text-align: left"> 812.775</td>
        <td style = "text-align: left"> 1.0</td>
        <td style = "text-align: left">2958511.5</td>
    </tr>
</table>

<html>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            سوالات:
        </b>
    </div>
    <br>
    <div dir = "rtl" style = "font-size: 16px">
           ۱- یک تابع 
        heuristic
        خوب باید به استیت هایی که شانس برد ما در آن ها بیشتر از امتیاز بیشتری اختصاص دهد. علت انتخاب 
        heuristic
        ذکر شده در بالا این است که این تابع جامعیت خوبی دارد و به حالات مختلف امتیازات مختلف می‌دهد. مثلا به سه مهره مجاور امتیاز بیشتری نسبت به دو مهره مجاور می‌دهد، و به دو مهره مجاور که جاهای خالی بیشتری در دو طرف آن است امتیاز بیشتری نسبت به دو مهره مجاور می‌دهد که فضای زیادی در اطراف آن ها خالی نیست.
        <br>
        از مزیت های این تابع نسبت به بقیه توابع می‌توان به این موضوع اشاره کرد که این تابع تنها به حالاتی امتیاز می‌دهد که ممکن است این حالت به پایان بازی ختم شود و این چینش پتانسیل برای چهارتایی شدن را دارد.
        <br><br>
        ۲- بلی رابطه وجود دارد: با افزایش عمق جست و جو شانس برد بیشتر می‌شود ولی زمان اجرا و تعداد راس های دیده شده نیز به صورت نمایی افزایش می‌یابد.
        <br><br>
        ۳- بلی این ترتیب اهمیت دارد چون می‌تواند از دیدن راس های زیادی جلوگیری کند، من در اینجا فرزندان یک راس را به صورت رندوم اضافه می‌کنم یعنی به صورت تصادفی انتخاب می‌کنم تا در کدام ستون مهره بریزم، دلیل این کار این  است که باعث میشود مهره ها در یک ستون جمع نشوند و حریف نتواند با یک حرکت آن را بلا استفاده کند، به عبارت دیگر مهره ها پخش می‌شوند و برای آینده حرکات موثر تری به وجود می‌اید.
        <br><br>
    </div> 
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            نتیجه گیری:
        </b>
    </div>
    <br>
    <div dir = "rtl" style = "font-size: 16px">
           برای افزایش شانس برد در یک بازی نیازی نیست تمام حالات را در نظر بگیریم و با یک استراتژی درست و تعریف 
        heuristic
        مناسب و استفاده از تکنیک هایی مانند هرس آلفا بتا می‌توان بدون چک کردن تمام حالات و در زمان قابل قبول یک استراتژی با شانس برد بالا اراعه داد.
    </div>
    <br><br>
    <div dir = "rtl" style="font-size: 22px; color: green">
        <b>
            پیشنهاد:
        </b>
    </div>
    <br>
    <div dir = "rtl" style = "font-size: 16px">
           اگر امکان تغییر کد اولیه وجود داشت، امکان این وجود داشت که کد تمیز تری در نهات به دست آید همچنین با افزودن قابلیت برگشت به بازی می‌توانستیم زمان اجرا را به مقدار قابل توجهی کاهش دهیم چون در این صورت استفاده از copy.deepcopy()
        کاهش پیدا می‌کرد که این متد بسیار زمان گیر است..
    </div>
    
</html>
